In [99]:
import gym
from gym import wrappers

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Reshape, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber, MeanSquaredError
#import tensorflow.keras.losses.huber as huber

import numpy as np
import pandas as pd
import random
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython import display
from time import sleep
from collections import deque
from PIL import Image
import cv2 as cv
import pickle

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import importlib

import wrappers
importlib.reload(wrappers)

<module 'wrappers' from 'h:\\IUBH_Master_AI\\RL_playing_pong\\wrappers.py'>

In [100]:
# To plot pretty figures
%matplotlib inline

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# To get smooth animations
import matplotlib.animation as animation
mpl.rc('animation', html='jshtml')

In [101]:
# all_envs = gym.envs.registry.values()
# env_ids = [env_spec.id for env_spec in all_envs]
# print(sorted(env_ids))

In [102]:
count_pong = 0
for env_spec in gym.envs.registry.values():

    if "Pong" in str(env_spec.id):
        print(str(env_spec) + "\n")
        count_pong +=1
    
print(count_pong)

EnvSpec(id='ALE/Pong-v5', entry_point='gym.envs.atari:AtariEnv', reward_threshold=None, nondeterministic=False, max_episode_steps=27000, order_enforce=True, autoreset=False, disable_env_checker=False, new_step_api=False, kwargs={'game': 'pong', 'obs_type': 'rgb', 'repeat_action_probability': 0.25, 'full_action_space': False, 'frameskip': 4}, namespace='ALE', name='Pong', version=5)

EnvSpec(id='ALE/Pong-ram-v5', entry_point='gym.envs.atari:AtariEnv', reward_threshold=None, nondeterministic=False, max_episode_steps=27000, order_enforce=True, autoreset=False, disable_env_checker=False, new_step_api=False, kwargs={'game': 'pong', 'obs_type': 'ram', 'repeat_action_probability': 0.25, 'full_action_space': False, 'frameskip': 4}, namespace='ALE', name='Pong-ram', version=5)

EnvSpec(id='Pong-v0', entry_point='gym.envs.atari:AtariEnv', reward_threshold=None, nondeterministic=False, max_episode_steps=10000, order_enforce=True, autoreset=False, disable_env_checker=False, new_step_api=False, kwa

In [103]:
count_pong = 0
for env_spec in gym.envs.registry.values():

    if str(env_spec.id).startswith("Pong"):
        print(str(env_spec) + "\n")
        count_pong +=1
    
print(count_pong)

EnvSpec(id='Pong-v0', entry_point='gym.envs.atari:AtariEnv', reward_threshold=None, nondeterministic=False, max_episode_steps=10000, order_enforce=True, autoreset=False, disable_env_checker=False, new_step_api=False, kwargs={'game': 'pong', 'obs_type': 'rgb', 'repeat_action_probability': 0.25, 'full_action_space': False, 'frameskip': (2, 5)}, namespace=None, name='Pong', version=0)

EnvSpec(id='PongDeterministic-v0', entry_point='gym.envs.atari:AtariEnv', reward_threshold=None, nondeterministic=False, max_episode_steps=100000, order_enforce=True, autoreset=False, disable_env_checker=False, new_step_api=False, kwargs={'game': 'pong', 'obs_type': 'rgb', 'repeat_action_probability': 0.25, 'full_action_space': False, 'frameskip': 4}, namespace=None, name='PongDeterministic', version=0)

EnvSpec(id='PongNoFrameskip-v0', entry_point='gym.envs.atari:AtariEnv', reward_threshold=None, nondeterministic=False, max_episode_steps=400000, order_enforce=True, autoreset=False, disable_env_checker=Fals

In [104]:
#ENV_NAME = "PongNoFrameskip-v4"
# EnvSpec(id='PongNoFrameskip-v4', entry_point='gym.envs.atari:AtariEnv', reward_threshold=None, nondeterministic=False, 
# max_episode_steps=400000, order_enforce=True, autoreset=False, disable_env_checker=False, new_step_api=False, 
# kwargs={'game': 'pong', 'obs_type': 'rgb', 'repeat_action_probability': 0.0, 'full_action_space': False, 'frameskip': 1}, 
# namespace=None, name='PongNoFrameskip', version=4)

#ENV_NAME = "Pong-v4"

ENV_NAME = "ALE/Pong-v5"
# EnvSpec(id='ALE/Pong-v5', entry_point='gym.envs.atari:AtariEnv', reward_threshold=None, nondeterministic=False, 
# max_episode_steps=27000, order_enforce=True, autoreset=False, disable_env_checker=False, new_step_api=False, 
# kwargs={'game': 'pong', 'obs_type': 'rgb', 'repeat_action_probability': 0.25, 'full_action_space': False, 'frameskip': 4}, namespace='ALE', name='Pong', version=5)

In [147]:
# create the environment 
env = gym.make(ENV_NAME, render_mode='human',full_action_space= False )
env = wrappers.PreProcessPongFrame(env)
env = wrappers.FrameStack(env, 4)
env.action_space.seed(42)
obs = env.reset()
#print(env.observation_space)
print(env.observation_space.low)
print(env.observation_space.high)


[[[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 ...

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]]
[[[255 255 255 255]
  [255 255 255 255]
  [255 255 255 255]
  ...
  [255 255 255 255]
  [255 255 255 255]
  [255 255 255 255]]

 [[255 255 255 255]
  [255 255 255 255]
  [255 255 255 255]
  ...
  [255 255 255 255]
  [255 255 255 255]
  [255 255 255 255]]

 [[255 255 255 255]
  [255 255 255 255]
  [255 255 255 255]
  ...
  [255 255 255 255]
  [255 255 255 255]
  [255 255 255 255]]

 ...

 [[255 255 255 255]
  [255 255 255 255]
  [255 255 255 255]
  ...
  [255 255 255 255]
  [255 255 255 255]
  [255 255 255 255

In [106]:
print(env.observation_space.shape)
print(env.action_space)

(84, 84, 4)
Discrete(6)


In [107]:
env.reward_range

(-inf, inf)

In [108]:
for _ in range(100):
    #observation, reward, done, info = env.step(3)
    observation, reward, done, info = env.step(env.action_space.sample())
    print(info, reward, done)
    print(type(observation))
    print(observation)    
    #print(observation._force()) # using ._force() on LazyFrames 
    # pil_image=Image.fromarray(observation)
    # pil_image.show()
    if done:
        observation = env.reset()

env.close()

{'lives': 0, 'episode_frame_number': 4, 'frame_number': 4} 0.0 False
<class 'wrappers.LazyFrames'>
{'lives': 0, 'episode_frame_number': 8, 'frame_number': 8} 0.0 False
<class 'wrappers.LazyFrames'>
{'lives': 0, 'episode_frame_number': 12, 'frame_number': 12} 0.0 False
<class 'wrappers.LazyFrames'>
{'lives': 0, 'episode_frame_number': 16, 'frame_number': 16} 0.0 False
<class 'wrappers.LazyFrames'>
{'lives': 0, 'episode_frame_number': 20, 'frame_number': 20} 0.0 False
<class 'wrappers.LazyFrames'>
{'lives': 0, 'episode_frame_number': 24, 'frame_number': 24} 0.0 False
<class 'wrappers.LazyFrames'>
{'lives': 0, 'episode_frame_number': 28, 'frame_number': 28} 0.0 False
<class 'wrappers.LazyFrames'>
{'lives': 0, 'episode_frame_number': 32, 'frame_number': 32} 0.0 False
<class 'wrappers.LazyFrames'>
{'lives': 0, 'episode_frame_number': 36, 'frame_number': 36} 0.0 False
<class 'wrappers.LazyFrames'>
{'lives': 0, 'episode_frame_number': 40, 'frame_number': 40} 0.0 False
<class 'wrappers.LazyFra

In [109]:

# actions: 
# 0 -> 
# 2 -> up
# 3 -> down
# 4 -> up
# 5 -> down


# env = gym.make(ENV_NAME, render_mode='human',full_action_space= False )
# env.action_space.seed(42)
# obs, info = env.reset(seed=42, return_info=True)
# for _ in range(4):
#     observation, reward, done, info = env.step(3)
#     #observation, reward, done, info = env.step(env.action_space.sample())
#     print(info, reward, done)

#     if done:
#         pass
#         observation, info = env.reset(return_info=True)

# env.close()

In [148]:
def clipped_mse(y_true, y_pred):
    mse = MeanSquaredError()
    loss = mse(y_true, y_pred)
    loss = tf.clip_by_value(loss, -1, 1)
    return loss


In [111]:
#https://stackoverflow.com/a/68739696
# custom tensorflow error functions

# new_arr = keras.losses.MSE(new_true, new_prediction) # keras example that needs to be ported to tensorflow

def rmsle(y_true, y_pred):
  msle = MeanSquaredLogarithmicError()
  return K.sqrt(msle(y_true, y_pred))

In [149]:

REPLAY_BUFFER_SIZE = 10_000 # according to Maxim Lapan
REPLAY_START_SIZE = 1_000 # 
REWARD_REPLAY_BUFFER_SIZE = 1000
BATCH_SIZE = 32
EPISODES = 2
EPSILON_START = 1
EPSILON_DECAY = 0.001
ESPILON_MIN = 0.01


input_shape=(84, 84, 1)
learning_rate=1e-2 # from Aurélien Geron better than 1e-3 https://github.com/ageron/handson-ml2/blob/master/18_reinforcement_learning.ipynb, Maxim Lapan: 1e-4
optimizer = Adam
huber_loss = Huber(reduction=tf.keras.losses.Reduction.SUM) # check the reduction!
loss = clipped_mse
#loss = huber_loss
#loss = "mse"   # check also "rmse"
gamma = 0.99 # used for Bellman approximation
AVG_SCORE_STEP = 100 # smoothed average over number of episodes
SYNC_TARGET_FREQ = 1000 # frequency for syncing model weights from the training to the target model 
LOGGING_FREQ = 100 # frequency of print statements



In [150]:
print(env.unwrapped.get_action_meanings())
# print(env.get_action_meanings())

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']


In [151]:
useful_actions = (0, 2 , 3, 4, 5) # action 1 does not ssem to be used
action = random.choice(useful_actions)
print(action)

2


In [152]:
def data_stats(batch_actions, batch_rewards, batch_dones):
    buffer_data = list(zip(batch_actions, batch_rewards, batch_dones))
    df = pd.DataFrame(buffer_data, columns =["action", "reward", "done"])
    num_neg_rewards = len(df[df["reward"] == -1])
    num_zero_rewards = len(df[df["reward"] == 0])
    num_pos_rewards = len(df[df["reward"] == 1])
    print(f"{num_neg_rewards + num_pos_rewards+ num_zero_rewards} rewards: positive rewards: {num_pos_rewards}, negative rewards: {num_neg_rewards}")

    return df, num_neg_rewards, num_zero_rewards, num_pos_rewards

In [153]:
n_actions = 6
input_size = (84,84,4)
model = Sequential(name= "DQN")
# model.add(keras.Input(shape=(4,)))   # required for calling model.summary()
# model.add(keras.Input(shape=(84, 84, 1)))  # 250x250 RGB images
model.add(Input(shape=input_size))
#model.add(Reshape((input_size))) #    expected shape=(None, 84, 84, 4), found shape=(84, 84, 4)
model.add(Conv2D(32, (8, 8), strides=(4, 4)))#, input_shape=input_size))
model.add(Activation('relu'))
model.add(Conv2D(64, (4, 4), strides=(2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), strides=(1, 1)))
model.add(Activation('relu'))
model.add(Flatten())
#model.add(Dense(512))
#model.add(Activation('relu'))
#model.add(Dense(n_actions))
#model.add(Activation('linear'))

model.build()
print(model.summary())

Model: "DQN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_39 (Conv2D)          (None, 20, 20, 32)        8224      
                                                                 
 activation_59 (Activation)  (None, 20, 20, 32)        0         
                                                                 
 conv2d_40 (Conv2D)          (None, 9, 9, 64)          32832     
                                                                 
 activation_60 (Activation)  (None, 9, 9, 64)          0         
                                                                 
 conv2d_41 (Conv2D)          (None, 7, 7, 64)          36928     
                                                                 
 activation_61 (Activation)  (None, 7, 7, 64)          0         
                                                                 
 flatten_13 (Flatten)        (None, 3136)              0       

In [154]:
def DeepQNetwork(input_size, n_actions, learning_rate, optimizer, loss):
    model = Sequential(name= "DQN")
    # model.add(keras.Input(shape=(4,)))   # required for calling model.summary()
    # model.add(keras.Input(shape=(84, 84, 1)))  # 250x250 RGB images
    model.add(Input(shape=input_size))  # input_size
    #model.add(Reshape((input_size))) #    expected shape=(None, 84, 84, 4), found shape=(84, 84, 4)
    model.add(Conv2D(32, (8, 8), strides=(4, 4)))#, input_shape=input_size))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (4, 4), strides=(2, 2)))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3), strides=(1, 1)))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Input(shape=(3136))) # important additional input layer before the Dense layer
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(n_actions))
    model.add(Activation('linear'))

    model.build()
    print(model.summary())

    model.compile(optimizer= optimizer(learning_rate=learning_rate), loss=loss)

    return model

In [158]:
import re

class DoubleDQNAgent:
    def __init__(self, state_size, action_space):
        self.input_size = state_size
        self.action_space = action_space
        self.n_actions = len(action_space)
        self.q_net = DeepQNetwork(self.input_size, self.n_actions, learning_rate, optimizer, loss)
        self.target_q_net = DeepQNetwork(self.input_size, self.n_actions, learning_rate, optimizer, loss)
        self.replay_buffer = deque(maxlen=REPLAY_BUFFER_SIZE)
        self.reward_replay_buffer = deque(maxlen=REWARD_REPLAY_BUFFER_SIZE)
        self.buffer_filename = 'replay_buffer.pkl'
        self.gamma = gamma
        self.epsilon = EPSILON_START 
        self.epsilon_decay = EPSILON_DECAY
        self.epsilon_min = ESPILON_MIN
        self.step_counter = 0

        self.update_target_network() # copy the q_net weights to target_q_net

        try:
            self.load_replay_buffer()
            print("Loaded replay buffer from disk.")
        except:
            print("Failed to load replay buffer from disk. Started to fill replay buffer.")
            state = env.reset()
            self.fill_replay_buffer(env, state)
            try:
                self.save_replay_buffer()
                print("Saved replay buffer to disk")
            except:
                print("Failed to save generated replay buffer!")


    def epsilon_greedy(self, state):
        if self.epsilon > self.epsilon_min:
            self.epsilon = self.epsilon - self.epsilon_decay
        else:
            pass
        
        if np.random.rand() < self.epsilon:
            return random.choice(self.action_space)
        else:
            norm_state = state._force()/255.0 # convert LazyFrame to array and normalize it
            norm_state = np.expand_dims(norm_state, axis=0)
            actions = self.q_net.predict(norm_state)
            #print(f"Actions: {actions}")
            action = np.argmax(actions)  # do I need to specify the axis? 
            #print("Chosen action: {action}")
            return action

    def remember(self, state, action, reward, new_state, done):
        self.replay_buffer.append((state, action, reward, new_state, done))
        return ### Do Python methods in classes need a return statement?

    def play_one_step(self, env, state):
        action = self.epsilon_greedy(state)
        new_state, reward, done, info = env.step(action)
        #print(f"Done: {done}")
        self.replay_buffer.append((state, action, reward, new_state, done))
        return state, action, reward, new_state, done, info

    def fill_replay_buffer(self, env, state):
        while len(self.replay_buffer) < REPLAY_START_SIZE:
            action = random.choice(self.action_space)
            new_state, reward, done, info = env.step(action)
            self.replay_buffer.append((state, action, reward, new_state, done))
            state = new_state
        return print("ReplayBuffer is filled!")

    def replay_buffer_stats(self):
        
        buffer_actions = [item[1] for item in self.replay_buffer]
        buffer_rewards = [item[2] for item in self.replay_buffer]
        buffer_dones = [item[4] for item in self.replay_buffer]
        results = data_stats(buffer_actions, buffer_rewards, buffer_dones)
        return results

    def fill_reward_replay_buffer(self):
        df, num_neg_rewards, num_zero_rewards, num_pos_rewards = self.replay_buffer_stats()
        #df, _,_,_ =self.replay_buffer_stats()
        reward_index = list(df.loc[(df.reward == -1) | (df.reward == 1)].index)
        for index in reward_index:
            self.reward_replay_buffer.append(self.replay_buffer[index])

        return reward_index



    def minibatch_stats(self, minibatch):
        
        buffer_actions = [item[1] for item in minibatch]
        buffer_rewards = [item[2] for item in minibatch]
        buffer_dones = [item[4] for item in minibatch]
        df, num_neg_rewards, num_zero_rewards, num_pos_rewards = data_stats(buffer_actions, buffer_rewards, buffer_dones)
        if (num_neg_rewards + num_pos_rewards) < 1:
            no_rewards = True
        else:
            no_rewards = False
        return df, num_neg_rewards, num_zero_rewards, num_pos_rewards, no_rewards


    def update_target_network(self):
        """
        Update the target Q network
        """
        self.target_q_net.set_weights(self.q_net.get_weights())

    
    def train(self, batch_size= BATCH_SIZE):
        minibatch = random.sample(self.replay_buffer, batch_size)

        df, = self.minibatch_stats(minibatch)
        #state_batch, action_batch, reward_batch, new_state_batch, done_batch = random.sample(self.replay_buffer, batch_size)
        # loop over batch
        #print(minibatch)
        #print(type(minibatch))
        #print(state_batch)
        #print(type(state_batch))
        state_batch = []
        next_state_batch = []
        targets = []
        for state, action, reward, next_state, done in minibatch:
            #print(f"Action: {action}, reward: {reward}, done: {done}")
            norm_state_array = state._force()/255.0 # convert LazyFrame to array and normalize it
            norm_state_array = np.expand_dims(norm_state_array,axis = 0)
            state_batch.append(norm_state_array)

            norm_next_state_array = state._force()/255.0 # convert LazyFrame to array and normalize it
            norm_next_state_array = np.expand_dims(norm_next_state_array, axis=0)
            #print(norm_next_state_array.shape)
            next_state_batch.append(norm_next_state_array) 

            # Main QNet is used to find the next action
            q_vals = self.q_net(norm_state_array)
            #print(f"q_vals: {q_vals}")
            next_action = np.argmax(q_vals, axis= 0) # TODO: check for amax and argmax!!!


            target_q_vals = self.target_q_net(norm_next_state_array)
            #print(target_q_vals)
            #print(target_q_vals.shape)
            target_q_max = np.amax(target_q_vals, axis= 1)
            #print(f"target_q_max: {target_q_max}")

            #rewards = np.copy(q_vals)
            if reward == 1:
                print(f"One point for me!")
            rewards = np.zeros(self.n_actions)
            rewards[next_action] = reward

            if done:
                target = rewards
            else:
                #target = reward + self.gamma * target_q_max
                target = rewards + self.gamma * target_q_max
            print(f"target: {target}")
            targets.append(target)

        state_batch = np.squeeze(state_batch)
        #print(f"State batch {state_batch.shape}")    
        state_batch_array = np.array(state_batch)
        #print(f"targets: {targets}")
        targets_array = np.array(targets, dtype=np.float32)
        targets_array = targets_array.reshape((BATCH_SIZE, -1))
        #targets_array = np.expand_dims(targets_array, axis=0)
        #print(targets_array.shape)
        #print(targets_array)

        self.q_net.train_on_batch(state_batch_array, targets_array)
        self.step_counter +=1
        print (f"Training step {self.step_counter}")
        #print(state_batch)  
        #print(state_batch.shape)
    
    def train_agent(self):

        version = 0 # versioning for model saving
        scores = []
        episodes = []
        avg_scores = []
        
        for i in range (EPISODES):
            done = False
            score = 0.0
            state = env.reset()

            while not done:


                state, action, reward, new_state, done, info = self.play_one_step(env, state)
                score += reward
                self.remember(state, action, reward, new_state, done) 
                state = new_state
                self.train()
                #self.step_counter += 1 # already updated in the self.train method

                # update the target_network regularly
                if self.step_counter % SYNC_TARGET_FREQ == 0:
                    self.update_target_network()
                    print(f"Step: {self.step_counter} -  updating target model")

                if self.step_counter % LOGGING_FREQ == 0:
                    print(f"Episode {i}/ {EPISODES}, epsilon: {self.epsilon}, score: {score}")



            scores.append(score)
            episodes.append(i)

            avg_score= np.mean(scores[-AVG_SCORE_STEP:])

            avg_scores.append(avg_score)

            print(f"Finished episode {i}/ {EPISODES}, epsilon: {self.epsilon}, score: {score}, avg. score: {avg_score}") 
        
        if score == 21:
            self.save_model(version)
            version +=1
            print(f"Episode {i}/ {EPISODES}, epsilon: {self.epsilon}, score: {score}, avg. score: {avg_score}") 

        return ### Do Python methods in classes need a return statement?


    def save_model(self, version):
        self.q_net.save((f"models/dqn_model_{version}"))
        self.q_net.save_weights((f"models/dqn_model_{version}/weights_{version}.h5"))

    def save_replay_buffer(self):
        outfile = open(self.buffer_filename,'wb')
        pickle.dump(self.replay_buffer,outfile)
        outfile.close()
        return print(f"Saved Replay Buffer to {self.buffer_filename} ")

    def load_replay_buffer(self):
        infile = open(self.buffer_filename,'rb')
        self.replay_buffer = pickle.load(infile)
        infile.close()
        return print(f"Loaded {self.buffer_filename} to Replay Buffer")


In [159]:
input_shape = env.observation_space.shape
print(input_shape)


(84, 84, 4)


In [160]:
agent = DoubleDQNAgent(input_shape, useful_actions)

Model: "DQN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_48 (Conv2D)          (None, 20, 20, 32)        8224      
                                                                 
 activation_72 (Activation)  (None, 20, 20, 32)        0         
                                                                 
 conv2d_49 (Conv2D)          (None, 9, 9, 64)          32832     
                                                                 
 activation_73 (Activation)  (None, 9, 9, 64)          0         
                                                                 
 conv2d_50 (Conv2D)          (None, 7, 7, 64)          36928     
                                                                 
 activation_74 (Activation)  (None, 7, 7, 64)          0         
                                                                 
 flatten_16 (Flatten)        (None, 3136)              0       

In [162]:
agent.replay_buffer_stats()

1000 rewards: positive rewards: 0, negative rewards: 21


(     action  reward   done
 0         3     0.0  False
 1         0     0.0  False
 2         2     0.0  False
 3         5     0.0  False
 4         3     0.0  False
 ..      ...     ...    ...
 995       2     0.0   True
 996       5     0.0   True
 997       5     0.0   True
 998       0     0.0   True
 999       3     0.0   True
 
 [1000 rows x 3 columns],
 21,
 979,
 0)

In [120]:
agent.load_replay_buffer()

Loaded replay_buffer.pkl to Replay Buffer


In [121]:
agent.train()

target: [0.13570976 0.13570976 0.13570976 0.13570976 0.13570976]
target: [0.13608041 0.13608041 0.13608041 0.13608041 0.13608041]
target: [0.13915834 0.13915834 0.13915834 0.13915834 0.13915834]
target: [0.1393107 0.1393107 0.1393107 0.1393107 0.1393107]
target: [0.13725951 0.13725951 0.13725951 0.13725951 0.13725951]
target: [0.13559259 0.13559259 0.13559259 0.13559259 0.13559259]
target: [0.14229918 0.14229918 0.14229918 0.14229918 0.14229918]
target: [0.13790828 0.13790828 0.13790828 0.13790828 0.13790828]
target: [0.13878377 0.13878377 0.13878377 0.13878377 0.13878377]
target: [0.13783324 0.13783324 0.13783324 0.13783324 0.13783324]
target: [0.14247653 0.14247653 0.14247653 0.14247653 0.14247653]
target: [0.14119501 0.14119501 0.14119501 0.14119501 0.14119501]
target: [0.14091191 0.14091191 0.14091191 0.14091191 0.14091191]
target: [0.13387497 0.13387497 0.13387497 0.13387497 0.13387497]
target: [0.13763298 0.13763298 0.13763298 0.13763298 0.13763298]
target: [0.13972247 0.13972247

In [122]:
agent.train_agent()

target: [0.13794863 0.13794863 0.13794863 0.13794863 0.13794863]
target: [0.13559259 0.13559259 0.13559259 0.13559259 0.13559259]
target: [0.13387497 0.13387497 0.13387497 0.13387497 0.13387497]
target: [0.12897466 0.12897466 0.12897466 0.12897466 0.12897466]
target: [0.1306302 0.1306302 0.1306302 0.1306302 0.1306302]
target: [0.14014851 0.14014851 0.14014851 0.14014851 0.14014851]
target: [0.13790929 0.13790929 0.13790929 0.13790929 0.13790929]
target: [0.13204329 0.13204329 0.13204329 0.13204329 0.13204329]
target: [0.14119501 0.14119501 0.14119501 0.14119501 0.14119501]
target: [0.13619521 0.13619521 0.13619521 0.13619521 0.13619521]
target: [0.13570976 0.13570976 0.13570976 0.13570976 0.13570976]
target: [0.13603258 0.13603258 0.13603258 0.13603258 0.13603258]
target: [0.13959266 0.13959266 0.13959266 0.13959266 0.13959266]
target: [0.14160801 0.14160801 0.14160801 0.14160801 0.14160801]
target: [0.13991539 0.13991539 0.13991539 0.13991539 0.13991539]
target: [0.1367414 0.1367414 0

KeyboardInterrupt: 

In [132]:
df, _,_,_ =agent.replay_buffer_stats()

100 rewards: positive rewards: 0, negative rewards: 2


In [142]:
reward_index = list(df.loc[(df.reward == -1) | (df.reward == 1)].index)
reward_index

[63, 98]

In [144]:
agent.replay_buffer[63]

(<wrappers.LazyFrames at 0x1c0cca950d0>,
 3,
 -1.0,
 False)

In [145]:
for index in reward_index:
    agent.reward_replay_buffer.append(agent.replay_buffer[index])

In [146]:
agent.reward_replay_buffer

deque([(<wrappers.LazyFrames at 0x1c0cca950d0>,
        3,
        -1.0,
        False),
       (<wrappers.LazyFrames at 0x1c0cca95df0>,
        2,
        -1.0,
        False)])

In [ ]:
agent.fill_replay_buffer(env, obs)

ReplayBuffer is filled!


In [ ]:
batch = random.sample(agent.replay_buffer, 32)
type(batch)
#state_batch, action_batch, reward_batch, new_state_batch, done_batch
#state, action, reward, new_state, done

list

TypeError: sequence index must be integer, not 'slice'

In [ ]:
batch[0]

(<wrappers.LazyFrames at 0x1bf3a6422e0>,
 3,
 0.0,
 False)

In [52]:
1 in reward_batch

False

In [97]:
buffer_actions = [item[1] for item in agent.replay_buffer]
buffer_rewards = [item[2] for item in agent.replay_buffer]
buffer_dones = [item[4] for item in agent.replay_buffer]

In [ ]:
action_batch = [item[1] for item in batch]
print(action_batch)
reward_batch =[[item[2] for item in batch]]
print(reward_batch)

[3, 0, 2, 4, 0, 5, 0, 3, 2, 0, 3, 0, 4, 5, 5, 0, 5, 3, 3, 4, 4, 3, 4, 0, 0, 5, 0, 5, 0, 2, 5, 0]
[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]


In [98]:
results = data_stats(buffer_actions, buffer_rewards, buffer_dones)

100 rewards: positive rewards: 0, negative rewards: 2


In [68]:
buffer_data = list(zip(buffer_actions, buffer_rewards, buffer_dones))
df = pd.DataFrame(buffer_data, columns =["action", "reward", "done"])
df

,action,reward,done
0,4,0.0,False
1,5,0.0,False
2,4,0.0,False
3,3,0.0,False
4,3,0.0,False
...,...,...,...
95,4,0.0,False
96,2,0.0,False
97,3,0.0,False
98,2,-1.0,False


In [93]:
num_neg_rewards = len(df[df["reward"] == -1])
num_zero_rewards = len(df[df["reward"] == 0])
num_pos_rewards = len(df[df["reward"] == 1])
print(f"{num_neg_rewards + num_pos_rewards+ num_zero_rewards} rewards: positive rewards: {num_pos_rewards}, negative rewards: {num_neg_rewards}")

100 rewards: positive rewards: 0, negative rewards: 2


In [ ]:
agent.train_agent()

target: [0.02900957 0.02900957 0.02900957 0.02900957 0.02900957]
target: [0.03778169 0.03778169 0.03778169 0.03778169 0.03778169]
target: [0.03148443 0.03148443 0.03148443 0.03148443 0.03148443]
target: [0.03029426 0.03029426 0.03029426 0.03029426 0.03029426]
target: [0. 0. 0. 0. 0.]
target: [0.02991573 0.02991573 0.02991573 0.02991573 0.02991573]
target: [0.0261976 0.0261976 0.0261976 0.0261976 0.0261976]
target: [0.03164728 0.03164728 0.03164728 0.03164728 0.03164728]
target: [0.03197697 0.03197697 0.03197697 0.03197697 0.03197697]
target: [0. 0. 0. 0. 0.]
target: [0.03120582 0.03120582 0.03120582 0.03120582 0.03120582]
target: [0.03320789 0.03320789 0.03320789 0.03320789 0.03320789]
target: [0.03298202 0.03298202 0.03298202 0.03298202 0.03298202]
target: [-0.96897466  0.03102534  0.03102534  0.03102534  0.03102534]
target: [0.03398627 0.03398627 0.03398627 0.03398627 0.03398627]
target: [0.02945234 0.02945234 0.02945234 0.02945234 0.02945234]
target: [0.03322248 0.03322248 0.0332224

KeyboardInterrupt: 

In [73]:
df.action.value_counts()

3    26
0    22
4    19
5    17
2    16
Name: action, dtype: int64

In [47]:
action_array = np.asarray(action_batch)


In [48]:
action_array

array([3, 0, 2, 4, 0, 5, 0, 3, 2, 0, 3, 0, 4, 5, 5, 0, 5, 3, 3, 4, 4, 3,
       4, 0, 0, 5, 0, 5, 0, 2, 5, 0])

In [35]:
obs = env.reset()
agent.epsilon_greedy(obs)

2

In [160]:
agent.train(BATCH_SIZE)

State batch (32, 84, 84, 4)
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[[[0.34117647 0.34117647 0.34117647 0.34117647]
   [0.34117647 0.34117647 0.34117647 0.34117647]
   [0.34117647 0.34117647 0.34117647 0.34117647]
   ...
   [0.34117647 0.34117647 0.34117647 0.34117647]
   [0.34117647 0.34117647 0.34117647 0.34117647]
   [0.34117647 0.34117647 0.34117647 0.34117647]]

  [[0.34117647 0.34117647 0.34117647 0.34117647]
   [0.34117647 0.34117647 0.34117647 0.34117647]
   [0.34117647 0.34117647 0.34117647 0.34117647]
   ...
   [0.34117647 0.34117647 0.34117647 0.34117647]
   [0.34117647 0.34117647 0.34117647 0.34117647]
   [0.34117647 0.34117647 0.34117647 0.34117647]]

  [[0.34117647 0.34117647 0.34117647 0.34117647]
   [0.34117647 0.34117647 0.34117647 0.34117647]
   [0.34117647 0.34117647 0.34117647 0.34117647]
   ...
   [0.34117647 0.34

In [50]:
agent.save_replay_buffer()

Saved Replay Buffer to replay_buffer.pkl 


In [33]:
agent.replay_buffer[0][3]

array([[[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [109, 118,  43],
        [109, 118,  43],
        [109, 118,  43]],

       [[109, 118,  43],
        [109, 118,  43],
        [109, 118,  43],
        ...,
        [109, 118,  43],
        [109, 118,  43],
        [109, 118,  43]],

       [[109, 118,  43],
        [109, 118,  43],
        [109, 118,  43],
        ...,
        [109, 118,  43],
        [109, 118,  43],
        [109, 118,  43]],

       ...,

       [[ 53,  95,  24],
        [ 53,  95,  24],
        [ 53,  95,  24],
        ...,
        [ 53,  95,  24],
        [ 53,  95,  24],
        [ 53,  95,  24]],

       [[ 53,  95,  24],
        [ 53,  95,  24],
        [ 53,  95,  24],
        ...,
        [ 53,  95,  24],
        [ 53,  95,  24],
        [ 53,  95,  24]],

       [[ 53,  95,  24],
        [ 53,  95,  24],
        [ 53,  95,  24],
        ...,
        [ 53,  95,  24],
        [ 53,  95,  24],
        [ 53,  95,  24]]

In [28]:
outfile = open('replay_buffer.pkl','wb')
pickle.dump(agent.replay_buffer,outfile)
outfile.close()
print(f"Saved Replay Buffer to {'replay_buffer.pkl'} ")

Saved Replay Buffer to replay_buffer.pkl 


In [ ]:
def preprocess_frame(image, shape=(84,84)):
    assert 
    image = image.astype(np.uint8) # required for cv2
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    cropped_image = image[34:34+160, :160] # remove scores
    cropped_image = cv2.resize(cropped_image, shape, interpolation = cv2.INTER_NEAREST) # INTER_AREA
    cropped_image= cropped_image.reshape((*shape, 1))
    return cropped_image



In [ ]:
observation_buffer  = deque(maxlen=4)


In [16]:
#from argparse import Action


env.seed(42)
obs = env.reset()

done = False
while not done:
    #env.render('human')
    #action = env.action_space.sample()
    action = random.choice(useful_actions)
    print(action)
    new_state, reward, done, info = env.step(action)

env.close()

4
5
4
0
4
4
4
4
2
3
4
2
2
3
5
4
2
4
0
3
5
3
2
0
0
4
0
4
3
0
0
2
0
5
3
5
4
5
5
4
3
0
2
5
3
0
3
0
5
3
2
5
2
4
2
0
4
4
4
2
2
4
0
5
5
0
3
2
5
3
4
0
3
5
0
4
0
5
4
5
4
3
3
5
2
5
0
5
3
4
5
0
0
2
0
2
4
0
4
5
5
0
3
0
3
3
5
4
2
0
3
3
2
0
0
5
5
2
2
4
0
0
0
0
0
4
2
4
2
3
5
5
5
5
2
4
2
0
4
0
2
0
4
0
4
3
4
4
4
2
0
5
3
4
5
3
0
3
2
5
0
4
2
0
0
2
4
2
3
5
5
2
4
4
0
0
2
3
4
0
2
0
2
4
4
0
0
3
5
4
4
5
0
5
2
2
5
3
2
2
4
2
0
5
4
3
5
4
2
4
3
4
4
3
5
3
5
4
5
3
4
4
3
0
2
0
3
3
2
3
4
5
0
2
2
2
4
0
4
5
4
4
2
4
4
4
3
2
3
2
5
0
2
0
5
4
4
0
2
5
5
4
4
3
5
5
2
4
4
2
3
2
3
2
5
4
5
5
3
3
4
5
2
0
0
5
4
5
2
0
3
2
5
4
3
2
5
5
3
3
0
4
2
2
3
0
0
2
3
0
4
0
3
4
0
3
0
3
0
0
3
5
4
0
0
5
3
0
3
3
4
3
5
5
4
3
2
4
5
2
5
5
2
3
5
0
4
2
2
3
2
2
4
2
4
5
3
4
4
2
2
3
4
2
5
2
3
2
2
3
2
5
4
4
0
2
3
5
4
0
0
2
2
0
4
4
2
3
2
3
3
2
3
5
5
4
5
5
3
2
2
0
0
2
5
3
3
5
3
3
3
2
2
0
2
5
2
4
3
5
5
0
3
5
3
2
3
4
3
2
5
0
5
4
5
5
3
3
3
0
0
4
5
3
4
2
4
0
3
3
2
5
0
2
4
4
5
2
0
5
2
5
3
4
4
3
3
5
3
2
3
2
2
2
5
3
5
2
4
2
3
4
4
4
0
0
4
3
2
2
5
2
4
3
3
2
3
4
5
0


In [ ]:
env.observation_space.shape

(210, 160, 3)

In [ ]:
env.action_space.n

6

In [ ]:
#env.render(mode='rgb_array')
    
